## Récupérer les coordonnées des centres de distribution

In [63]:
import pandas as pd

df = pd.read_excel("data/centres.xlsx")

In [64]:
names_centres = df["Nom"].tolist()
adresses_centres = df["Adresse"].tolist()

In [65]:
import requests

api_key = open("api_key", "r").read()

In [66]:
def get_coords(addresses):
    latitudes = []
    longitudes = []

    for address in addresses:
        response = requests.get(
            f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
        )
        data = response.json()

        if data["status"] == "OK":
            location = data["results"][0]["geometry"]["location"]
            latitudes.append(location["lat"])
            longitudes.append(location["lng"])
        else:
            print(f"Geocoding failed for address {address}. Status:", data["status"])
    
    return latitudes, longitudes

In [68]:

latitudes, longitudes = get_coords(adresses_centres)

In [67]:
# for n, lat, long in zip(names_centres, latitudes, longitudes) :
#     print(    f"{n} : {lat}, {long}")

In [35]:
# Add the latitudes as new columns into the original excel file
df["Latitude"] = latitudes
df["Longitude"] = longitudes

In [36]:
df.to_excel("data/centres.xlsx", index=False)

## Récupérer les coordonnées des points de ramasse

In [70]:
pdr = pd.read_excel("data/points_de_ramasse.xlsx")

In [76]:
adresses_pdr = pdr["Adresse"].tolist()
# latitudes, longitudes = get_coords(adresses)

In [11]:
for lat, long in zip(latitudes, longitudes) :
    print(    f"{lat}, {long}")

43.6431035, 1.4160821
43.6319261, 1.4843222
43.5661466, 1.5171659
43.5864191, 1.5776498
43.644588, 1.3726111
43.6666046, 1.514161
43.5913663, 1.2960563


In [12]:
pdr["Latitude"] = latitudes
pdr["Longitude"] = longitudes
pdr.to_excel("data/centres.xlsx", sheet_name="Points de Ramasse", index=False)

## Créer la matrice de distances

In [72]:
import googlemaps

gmaps = googlemaps.Client(key=api_key)

In [77]:
from tqdm import tqdm

matrix = []
adresses = adresses_centres + adresses_pdr

for i, a in tqdm(enumerate(adresses), total=len(adresses)):
    matrix.append([])
    for j, b in enumerate(adresses):
        if i==j :
            matrix[i].append(0)
            continue
        
        results = gmaps.directions(a, b, alternatives=True,)
        results += gmaps.directions(a, b, alternatives=True, avoid="highways",)
        results += gmaps.directions(a, b, alternatives=True, avoid="tolls",)
        results += gmaps.directions(a, b, alternatives=True, avoid="tolls|highways",)
        dist = min(
            res["legs"][0]["distance"]["value"] for res in results
        )
        matrix[i].append(dist)

100%|██████████| 36/36 [08:51<00:00, 14.78s/it]


In [78]:
def make_euclidean(matrix):
    corrections = 0
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if j == i:
                continue
            for k in range(len(matrix)):
                if k == i or k == j:
                    continue
                if matrix.iloc[i, j] + matrix.iloc[j, k] < matrix.iloc[i, k]:
                    corrections += 1
                    matrix.iloc[i, k] = matrix.iloc[i, j] + matrix.iloc[j, k]
    print("Failures : ", corrections)

In [79]:
noms = df["Nom"].tolist() + pdr["Nom"].tolist()
mat = pd.DataFrame(matrix, columns=noms, index=noms)
mat.to_excel("data/non_euclidean_matrix.xlsx")
make_euclidean(mat)
mat.to_excel("data/euclidean_matrix.xlsx")

Failures :  610
